# Wave Download Memory Test

### Description:
This purpose of this script is to check the memory size. 16 waveforms of 1G samples have been downloaded to the memory to check the 16G memory size.


Proteus release - Aurora

Note: For accurate time readings DLL connection is preferred with tabor proteus services turned off, No other connection required to run this script.

In [1]:
# slot_id = '2'
ip_addr='192.90.70.22'
max = 0
channb = 1 #input("Channel number to test the waveform speed")

In [2]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

# Connect to instrument
inst = connect('192.90.70.22')
# inst = connect("2",Auto=False)
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))
# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':TRACe:DELete:ALL')
# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
init_arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(init_arbmem_capacity))

Service connect
Trying to connect to IP:192.90.70.22
Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M 
DAC 16 bits
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes


In [3]:
## Configure Generator
print("start")
# for i in np.arange(0,16,1):
segnum = 0
arbmem_capacity_after_download = init_arbmem_capacity
while(arbmem_capacity_after_download>0):
    segnum = segnum +1
    seglen = 268304384#268304384*2 #102400000
    num_cycles=1
    cyclelen = seglen / num_cycles
    # Build waveform
    x = np.linspace(start=0, stop=2 * np.pi * num_cycles, num=seglen, endpoint=False)
    y = (np.sin(x) + 1.0) * half_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        wave = y.astype(np.uint16)
    else:
        wave = y.astype(np.uint8)

    # data-type code: 'B' = uint8, 'H' = uint16
    data_type = 'H' if dac_mode == 16 else 'B'

# for loop iterating over the range of sampling rate from 1GHz to 9Ghz with a step of .5GHz

    inst.send_scpi_cmd(":FREQ:RAST 2500MHz")
    resp = inst.send_scpi_query(":FREQ:RAST?")
    freq = float(resp)/1e6
    #print ("Press enter to download the waveform for sampling rate :{0}MHz".format(freq))
    #input()
    inst.timeout = 30000
    cmd = ':INST:CHAN {0}'.format(channb)
    inst.send_scpi_cmd(cmd)

    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    inst.send_scpi_cmd(cmd)

    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    inst.timeout = 30000

    #start_time = time.time()
    tic = time.perf_counter()

    inst.write_binary_data(':TRAC:DATA', wave)

    #end_time = time.time()
    #time_lapsed = end_time - start_time

    #print('Downloaded the wave in "{0}" seconds'.format(time_lapsed))

    
    toc = time.perf_counter()
    timer = float(toc-tic)
    print('Segnum = {0},Downloaded'.format(segnum))


    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
         print('ERROR: "{0}" after writing binary values'.format(resp))

    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)
    resp = inst.send_scpi_query(":TRACe:FREE?")
    arbmem_capacity_after_download = int(resp)
#     print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity_after_download))
    if arbmem_capacity_after_download == 0:
        print("Maximum segment downloads = {}".format(segnum))
        
    x = int(init_arbmem_capacity - arbmem_capacity_after_download) 
    print('memory occupied ',x)


start
Segnum = 1,Downloaded


VisaIOError: VI_ERROR_CONN_LOST (-1073807194): The connection for the given session has been lost.

In [ ]:
disconnect()    
print('Test Completed')